# LangChain with Open Source LLM and Open Source Embeddings & LangSmith

In the following notebook we will dive into the world of Open Source models hosted on Hugging Face's [inference endpoints](https://ui.endpoints.huggingface.co/).

The notebook will be broken into the following parts:

- 🤝 Breakout Room #1:
  1. Set-up Hugging Face Infrence Endpoints
  2. Install required libraries
  3. Set Environment Variables
  4. Testing our Hugging Face Inference Endpoint
  5. Creating LangChain components powered by the endpoints
  6. Retrieving data from Arxiv
  7. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Set-up LangSmith
  2. Creating a LangSmith dataset
  3. Creating a custom evaluator
  4. Initializing our evaluator config
  5. Evaluating our RAG pipeline

# 🤝 Breakout Room #1

## Task 1: Set-up Hugging Face Infrence Endpoints

Please follow the instructions provided [here](https://github.com/AI-Maker-Space/AI-Engineering-Cohort-2/tree/main/Week%205/Day%202) to set-up your Hugging Face inference endpoints for both your LLM and your Embedding Models.

## Task 2: Install required libraries

Now we've got to get our required libraries!

We'll start with our `langchain` and `huggingface` dependencies.



In [1]:
!pip install langchain langchain-core langchain-community langchain_openai huggingface-hub requests -q -U

Now we can grab some miscellaneous dependencies that will help us power our RAG pipeline!

In [2]:
!pip install arxiv pymupdf faiss-cpu -q -U

In [3]:
!pip install numpy==1.21.6

## Task 3: Set Environment Variables

We'll need to set our `HF_TOKEN` so that we can send requests to our protected API endpoint.

We'll also set-up our OpenAI API key, which we'll leverage later.



In [4]:
import os
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("HuggingFace Write Token: ")

HuggingFace Write Token: ··········


In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


## Task 4: Testing our Hugging Face Inference Endpoint

Let's submit a sample request to the Hugging Face Inference endpoint!

In [9]:
model_api_gateway = "https://w00ju9fjxc2vzppr.us-east-1.aws.endpoints.huggingface.cloud" # << YOUR ENDPOINT URL HERE

> NOTE: If you're running into issues finding your API URL you can find it at [this](https://ui.endpoints.huggingface.co/) link.

Here's an example:

![image](https://i.imgur.com/XyZhOv8.png)

In [10]:
import requests

max_new_tokens = 256
top_p = 0.9
temperature = 0.1

prompt = "Hello! How are you?"

json_body = {
    "inputs" : prompt,
    "parameters" : {
        "max_new_tokens" : max_new_tokens,
        "top_p" : top_p,
        "temperature" : temperature
    }
}

headers = {
  "Authorization": f"Bearer {os.environ['HF_TOKEN']}",
  "Content-Type": "application/json"
}

response = requests.post(model_api_gateway, json=json_body, headers=headers)
print(response.json())

[{'generated_text': "Hello! How are you? I'm doing well, thanks for asking! *smiling*\n\n\n"}]


## Task 5: Creating LangChain components powered by the endpoints

We're going to wrap our endpoints in LangChain components in order to leverage them, thanks to LCEL, as we would any other LCEL component!

### HuggingFaceEndpoint for LLM

We can use the `HuggingFaceEndpoint` found [here](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/huggingface_endpoint.py) to power our chain - let's look at how we would implement it.

In [11]:
from langchain.llms import HuggingFaceEndpoint

endpoint_url = (
    model_api_gateway
)

hf_llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
    task="text-generation"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Now we can use our endpoint like we would any other LLM!

In [12]:
hf_llm.invoke("Hello, how are you?")

'\n\nYes, I am a big fan of your work and I must say that I am very impressed by the way you have been able to blend different styles and create something unique and interesting. Your use of color and light is particularly impressive and it really adds depth and emotion to your work.\n\nI was wondering if you could tell me a little bit about your creative process and how you come up with your ideas? Do you have any specific techniques or rituals that you use to get into a creative state of mind?\n\nAlso, I was curious to know if you have any upcoming exhibitions or projects that you are working on? I would love to hear more about them.\n\nThank you so much for your time and I look forward to hearing back from you.\n\nBest regards,\n[Your Name]'

### HuggingFaceInferenceAPIEmbeddings

Now we can leverage the `HuggingFaceInferenceAPIEmbeddings` module in LangChain to connect to our Hugging Face Inference Endpoint hosted embedding model.

In [13]:
embedding_api_gateway = "https://drk3aaxtilds5moy.us-east-1.aws.endpoints.huggingface.cloud" # << Embedding Endpoint API URL

In [14]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings_model = HuggingFaceInferenceAPIEmbeddings(api_key=os.environ["HF_TOKEN"], api_url=embedding_api_gateway)

In [15]:
embeddings_model.embed_query("Hello, welcome to HF Endpoint Embeddings")[:10]

[-0.026445465,
 0.03584882,
 0.009444279,
 0.011648565,
 0.006504414,
 0.008241854,
 -0.036926128,
 -0.036308434,
 -0.02483537,
 -0.0053980113]

#### ❓ Question #1

What is the embedding dimension of your selected embeddings model?

## Task 6: Retrieving data from Arxiv

We'll leverage the `ArxivLoader` to load some papers about the "QLoRA" topic, and then split them into more manageable chunks!

In [16]:
from langchain.document_loaders import ArxivLoader

docs = ArxivLoader(query="QLoRA", load_max_docs=5).load()

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap = 0,
    length_function = len,
)

split_chunks = text_splitter.split_documents(docs)

In [18]:
len(split_chunks)

1305

Just the same as we would with OpenAI's embeddings model - we can instantiate our `FAISS` vector store with our documents and our `HuggingFaceEmbeddings` model!

We'll need to take a few extra steps, though, due to a few limitations of the endpoint/FAISS.

We'll start by embeddings our documents in batches of `32`.

> NOTE: This process might take a while depending on the compute you assigned your embedding endpoint!

In [19]:
embeddings = []
for i in range(0, len(split_chunks) - 1, 32):
  embeddings.append(embeddings_model.embed_documents([document.page_content for document in split_chunks[i:i+32]]))

In [20]:
embeddings = [item for sub_list in embeddings for item in sub_list]

#### ❓ Question #2

Why do we have to limit our batches when sending to the Hugging Face endpoints?

Now we can create text/embedding pairs which we want use to set-up our FAISS VectorStore!

In [23]:
import numpy as np

# List to hold valid text and embedding pairs
valid_text_embedding_pairs = []

# Iterate through the documents and their corresponding embeddings
for document, emb in zip(split_chunks, embeddings):
    try:
        # Ensure embedding is a numpy array and convert it to float32
        emb_array = np.array(emb, dtype=np.float32)
        # Append the valid document content and embedding to the list
        valid_text_embedding_pairs.append((document.page_content, emb_array))
    except ValueError:
        # Handle cases where conversion to float array fails
        print(f"Failed to convert embedding for document: {document.page_content}")
        # Optionally, handle the error by continuing, logging, or using a placeholder embedding
        # Example of using a placeholder:
        # placeholder_embedding = np.zeros_like(valid_text_embedding_pairs[0][1])  # Assuming at least one valid embedding exists
        # valid_text_embedding_pairs.append((document.page_content, placeholder_embedding))

# Check if there are any valid embeddings left
if not valid_text_embedding_pairs:
    raise ValueError("No valid embeddings available.")

# Create the FAISS vector store with the valid pairs
faiss_vectorstore = FAISS.from_embeddings(valid_text_embedding_pairs, embeddings_model)


Failed to convert embedding for document: Dettmers and Zettlemoyer [13] and measure post-quantization zero-shot accuracy and perplexity
across different models (OPT [72], LLaMA [57], BLOOM [52], Pythia [7]) for model sizes 125m -
Failed to convert embedding for document: 13B (11.3 GB)
33B (24.7 GB)
65B (45.0 GB)
Input gradient
Optimizer
Weight gradient
Adapters
Model
Figure 6: Breakdown of the memory footprint of different LLaMA models. The input gradient size is for batch
Failed to convert embedding for document: size 1 and sequence length 512 and is estimated only for adapters and the base model weights (no attention).
Numbers on the bars are memory footprint in MB of individual elements of the total footprint. While some
Failed to convert embedding for document: models do not quite fit on certain GPUs, paged optimzier provide enough memory to make these models fit.
G
Memory Footprint
The memory footpring for QLoRA training with different LLaMA base models can be seen in
Failed to co

Next, we set up FAISS as a retriever.

In [24]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k" : 5})

Let's test it out!

In [25]:
faiss_retriever.get_relevant_documents("What optimizer does QLoRA use?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='We have discussed how QLoRA works and how it can significantly reduce the required memory for\nfinetuning models. The main question now is whether QLoRA can perform as well as full-model'),
 Document(page_content='performance degradation. Our method, QLORA, uses a novel high-precision technique to quantize\na pretrained model to 4-bit, then adds a small set of learnable Low-rank Adapter weights [28]\n∗Equal contribution.'),
 Document(page_content='wFP16 | w) denotes the conditional entropy of ˆ\nwFP16\ngiven w. As deterministic quantizers are used in the quan-\ntization of LLMs, H( ˆ\nwFP16 | w) = 0 and the I( ˆ\nwFP16; w)\ndepends on H( ˆ\nwFP16) directly. In the PTQ, since the origi-'),
 Document(page_content='A dataset for diverse, explainable multi-hop question answering. In Proceedings of the 2018\nConference on Empirical Methods in Natural Language Processing, pages 2369–2380, 2018.'),
 Document(page_content='ders information recovery. LoRA mitigates the p

### Prompt Template

Now that we have our LLM and our Retiever set-up, let's connect them with our Prompt Template!

In [26]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT_TEMPLATE = """\
Using the provided context, please answer the user's question. If you don't know, say you don't know.

Context:
{context}

Question:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

#### ❓ Question #3

Does the ordering of the prompt matter?



## Task 7: Creating a simple RAG pipeline with LangChain v0.1.0

All that's left to do is set up a RAG chain - and away we go!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

retrieval_augmented_qa_chain = (
    {
        "context": itemgetter("question") | faiss_retriever,
        "question": itemgetter("question"),
    }
    | rag_prompt
    | hf_llm
    | StrOutputParser()
)

Let's test it out!

In [28]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLORA all about?"})

'\nAnswer:\nQLoRA is a method for reducing the memory required for finetuning large language models. It does this by transforming the weights of the model into a lower-rank matrix, which can be stored in less memory. The method has been shown to be effective in reducing the memory required for finetuning, but it is not clear how well it performs compared to full-model finetuning.'

# 🤝 Breakout Room #2

## Task 1: Set-up LangSmith

We'll be moving through this notebook to explain what visibility tools can do to help us!

Technically, all we need to do is set-up the next cell's environment variables!

In [31]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE2 - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Enter your LangSmith API key: ··········


Let's see what happens on the LangSmith project when we run this chain now!

In [32]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLoRA all about?"})

'\nAnswer:\nQLoRA is a method for quantizing low-rank approximations of neural network weights. It is designed to reduce the memory required for finetuning pre-trained models, and can perform as well as full-model quantization.'

We get *all of this information* for "free":

![image](https://i.imgur.com/8Wcpmcj.png)

> NOTE: We'll walk through this diagram in detail in class.

####🏗️ Activity #1:

Please describe the trace of the previous request and answer these questions:

1. How many tokens did the request use?
2. How long did the `HuggingFaceEndpoint` take to complete?

## Task 2: Creating a LangSmith dataset

Now that we've got LangSmith set-up - let's explore how we can create a dataset!

First, we'll create a list of questions!

In [33]:
from langsmith import Client

questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

Now we can create our dataset through the LangSmith `Client()`.

In [34]:
client = Client()
dataset_name = "QLoRA RAG Dataset v2"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    dataset_id=dataset.id
)

After this step you should be able to navigate to the following dataset in the LangSmith web UI.

![image](https://i.imgur.com/CdFYGTB.png)

## Task 3: Creating a custom evaluator

Now that we have a dataset - we can start thinking about evaluation.

We're going to make a `StringEvaluator` to measure "dopeness".

> NOTE: While this is a fun toy example - this can be extended to practically any use-case!

In [35]:
import re
from typing import Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.evaluation import StringEvaluator

class DopenessEvaluator(StringEvaluator):
    """An LLM-based dopeness evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model="gpt-4", temperature=0)

        template = """On a scale from 0 to 100, how dope (cool, awesome, lit) is the following response to the input:
        --------
        INPUT: {input}
        --------
        OUTPUT: {prediction}
        --------
        Reason step by step about why the score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = PromptTemplate.from_template(template) | llm

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "scored_dopeness"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain.invoke(
            {"input": input, "prediction": prediction}, kwargs
        )
        reasoning, score = evaluator_result.content.split("\n", maxsplit=1)
        score = re.search(r"\d+", score).group(0)
        if score is not None:
            score = float(score.strip()) / 100.0
        return {"score": score, "reasoning": reasoning.strip()}

## Task 4: Initializing our evaluator config

Now we can initialize our `RunEvalConfig` which we can use to evaluate our chain against our dataset.

> NOTE: Check out the [documentation](https://docs.smith.langchain.com/evaluation/faq/custom-evaluators) for adding additional custom evaluators.

In [36]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[DopenessEvaluator()],
    evaluators=[
        "criteria",
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria(
            {
                "AI": "Does the response feel AI generated?"
                "Response Y if they do, and N if they don't."
            }
        ),
    ],
)

## Task 5: Evaluating our RAG pipeline

All that's left to do now is evaluate our pipeline!

In [37]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=retrieval_augmented_qa_chain,
    evaluation=eval_config,
    verbose=True,
    project_name="HF RAG Pipeline - Evaluation - v1",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'HF RAG Pipeline - Evaluation - v1' at:
https://smith.langchain.com/o/bbdaa341-a469-5436-ba9e-24733ea4fe6d/datasets/2bb9ebca-0ccc-4802-a664-20aa1bd62426/compare?selectedSessions=ca07b9f1-7fe5-4fa7-ab6f-7853eab0bef4

View all tests for Dataset QLoRA RAG Dataset v2 at:
https://smith.langchain.com/o/bbdaa341-a469-5436-ba9e-24733ea4fe6d/datasets/2bb9ebca-0ccc-4802-a664-20aa1bd62426
[------------------------------------------------->] 6/6
 Experiment Results:
        feedback.helpfulness  feedback.harmfulness  feedback.AI  feedback.scored_dopeness error  execution_time                                run_id
count                   6.00                  6.00         6.00                      6.00     0            6.00                                     6
unique                   NaN                   NaN          NaN                       NaN     0             NaN                                     6
top                      NaN                   NaN 

{'project_name': 'HF RAG Pipeline - Evaluation - v1',
 'results': {'135b4bee-07a7-407d-9038-2408270201e9': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='helpfulness', score=0, value='N', comment='The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate. \n\nLooking at the submission, the respondent simply states "I don\'t know." This does not provide any helpful or insightful information about the question asked, which is about the optimizer used in QLoRA. \n\nThe response is appropriate in the sense that it is a valid response to a question, but it does not meet the criteria of being helpful or insightful. \n\nTherefore, the submission does not meet the criteria.\n\nN', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('0b3c1681-8cfc-4824-9586-bb57efd603db'))}, feedback_config=None, source_run_id=None, target_run_id=None),
    EvaluationResult(key='harmfulness', score=0, 